# 🎵 Music Information Retrieval (MIR) Ultimate Analyzer [nnAudio GPU EDITION]
### *Powered by nnAudio, PyTorch (CUDA) & Librosa*

Este cuaderno ha sido diseñado para la máxima velocidad de procesamiento utilizando **GPU** a través de la librería **nnAudio**. A diferencia de Librosa, que es CPU-bound, nnAudio utiliza PyTorch para realizar transformadas de Fourier (STFT, CQT, Mel) directamente en la tarjeta de video, permitiendo el análisis masivo de datasets en una fracción del tiempo.

---

In [ ]:
!pip install -q nnAudio librosa torch pandas matplotlib tqdm

import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import torch
import shutil
import pandas as pd
from nnAudio import features
from IPython.display import Audio, display, FileLink
from tqdm.auto import tqdm

# Configuración de Dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 Motor de GPU: {device.upper()}")
if device == "cuda":
    print(f"Placa: {torch.cuda.get_device_name(0)}")

# Configuración Estética Premium
plt.style.use('dark_background')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (15, 6), 'lines.linewidth': 1.5})

OUTPUT_DIR = 'mir_nnaudio_results'
PLOTS_DIR = os.path.join(OUTPUT_DIR, 'plots')
os.makedirs(PLOTS_DIR, exist_ok=True)

## 📂 1. Preparación del Dataset y Exclusión
Configuramos el acceso a los archivos de Kaggle e ignoramos el archivo de referencia.

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
IGNORE_FILE = 'REF.flac'

if not os.path.exists(DATASET_PATH):
    print(f"⚠️ Path Kaggle no detectado. Cambiando a local: Slavic Data_Set")
    DATASET_PATH = 'Slavic Data_Set'

all_files = [f for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav', '.flac')) and f != IGNORE_FILE]
print(f"📚 Total de pistas a procesar vía GPU: {len(all_files)}")

## ⚡ 2. Inicialización de Capas nnAudio (GPU Kernels)
Creamos las capas de procesamiento que vivirán en la GPU.

In [ ]:
# Definimos capas de nnAudio fuera del bucle para máximo rendimiento
sr_default = 44100 # Se ajustará dinámicamente si es necesario

spec_layer = features.STFT(n_fft=2048, hop_length=512).to(device)
mel_layer = features.MelSpectrogram(sr=sr_default, n_fft=2048, n_mels=128).to(device)
cqt_layer = features.CQT2010v2(sr=sr_default, hop_length=512, fmin=32.7, n_bins=84).to(device)

print("✅ Kernels de nnAudio inicializados en GPU.")

## 🧬 3. El Algoritmo de Alto Rendimiento
Esta función coordina la carga de audio (CPU) y el análisis espectral masivo (GPU).

In [ ]:
def process_with_nnaudio(file_path, save_plots=True):
    filename = os.path.basename(file_path)
    
    # Carga rápida (Librosa sigue siendo necesario para decodificar MP3/FLAC)
    y, sr = librosa.load(file_path, sr=None, mono=True)
    y_torch = torch.from_numpy(y).float().to(device).unsqueeze(0)
    
    # --- ANALISIS GPU CON nnAudio ---
    with torch.no_grad():
        # Espectrograma de Mel
        melspec = mel_layer(y_torch)
        melspec_db = librosa.power_to_db(melspec.cpu().squeeze().numpy(), ref=np.max)
        
        # Transformada Constant-Q (Mucho más rápida en GPU)
        cqt_spec = cqt_layer(y_torch)
        cqt_db = librosa.amplitude_to_db(torch.abs(cqt_spec).cpu().squeeze().numpy(), ref=np.max)
    
    # --- ANALISIS COMPLEMENTARIO (Métricas de Tonalidad/Ritmo) ---
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)
    
    # Key Estimation
    major_p = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
    minor_p = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    major_corr = [np.corrcoef(chroma_mean, np.roll(major_p, i))[0, 1] for i in range(12)]
    minor_corr = [np.corrcoef(chroma_mean, np.roll(minor_p, i))[0, 1] for i in range(12)]
    key = f"{notes[np.argmax(major_corr)]} Major" if max(major_corr) > max(minor_corr) else f"{notes[np.argmax(minor_corr)]} Minor"

    # --- GUARDADO DE RESULTADOS ---
    if save_plots:
        fig, ax = plt.subplots(2, 1, figsize=(12, 10))
        img1 = librosa.display.specshow(melspec_db, ax=ax[0], x_axis='time', y_axis='mel', sr=sr, cmap='magma')
        ax[0].set(title=f'Mel Spectrogram (nnAudio): {filename}')
        plt.colorbar(img1, ax=ax[0])
        
        img2 = librosa.display.specshow(cqt_db, ax=ax[1], x_axis='time', y_axis='cqt_note', sr=sr, cmap='inferno')
        ax[1].set(title=f'CQT Spectrogram (nnAudio): {filename}')
        plt.colorbar(img2, ax=ax[1])
        
        plt.tight_layout()
        plt.savefig(os.path.join(PLOTS_DIR, f"{filename}_analysis.png"))
        plt.close()

    return {
        "filename": filename,
        "tempo_bpm": tempo,
        "key": key,
        "duration_sec": len(y)/sr,
        "rms_mean": np.mean(librosa.feature.rms(y=y))
    }

## 🚀 4. Lanzamiento del Análisis Masivo
Ejecutamos el motor sobre todo el dataset. El uso de nnAudio en la GPU verá una mejora significativa en archivos grandes.

In [ ]:
results = []
print("🚀 Motor nnAudio Iniciado...")

for f in tqdm(all_files, desc="Procesando Dataset"):
    path = os.path.join(DATASET_PATH, f)
    try:
        res = process_with_nnaudio(path)
        results.append(res)
    except Exception as e:
        print(f"❌ Error en {f}: {e}")

df = pd.DataFrame(results)
df.to_csv(os.path.join(OUTPUT_DIR, 'mir_gpu_report.csv'), index=False)
print(f"✅ Análisis completado. Reporte generado en {OUTPUT_DIR}/mir_gpu_report.csv")
display(df.head())

## 📦 5. Empaquetado de Resultados y Descarga Automática

In [ ]:
ZIP_NAME = 'MIR_nnAudio_GPU_Results.zip'

print("📦 Creando archivo ZIP final...")
if os.path.exists(ZIP_NAME):
    os.remove(ZIP_NAME)

shutil.make_archive('MIR_nnAudio_GPU_Results', 'zip', OUTPUT_DIR)

print(f"\n🎉 TODO LISTO: {ZIP_NAME}")
display(FileLink(ZIP_NAME, result_html_prefix="🔗 DESCARGA TU ANÁLISIS AQUÍ: "))